## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [1]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [2]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## 加载音频处理库

In [1]:
!pip install librosa --user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 183 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 613 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 1.8 MB/s  eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=f57d570bf49fa0e2e2c23bc32c6aa915e7935f2ef6c0b0ed1fb643d75c2eaf9a
  Stored in directory: /Users/sampras/Library/Caches/pip/wheels/c6/08/26/9d7dec1c2dc863d8f1fdc1ac39883e2601dc6fa63fcd68183f
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23142 sha256=1cc9ce52b5227f39fda38261e64321847cf1cbd17bb713b6ce633ae151e7005c
  Stored in directory: /Users/sampras/Library/Caches/pip/wheels/71/76/34/82f346dbd557abb79d415dd4c55281db62cca581ceb1099e0f
  Created wheel for resampy: filename=re

In [3]:
# 其他库

import os
import librosa
import librosa.display
import glob

## 特征提取以及数据集的建立

In [5]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {
    'aloe': 0,
    'burger': 1,
    'cabbage': 2,
    'candied_fruits': 3,
    'carrots': 4,
    'chips': 5,
    'chocolate': 6,
    'drinks': 7,
    'fries': 8,
    'grapes': 9,
    'gummies': 10,
    'ice-cream': 11,
    'jelly': 12,
    'noodles': 13,
    'pickles': 14,
    'pizza': 15,
    'ribs': 16,
    'salmon': 17,
    'soup': 18,
    'wings': 19
}
label_dict_inv = {v: k for k, v in label_dict.items()}

In [21]:
from tqdm import tqdm


def extract_features(parent_dir, sub_dirs, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(
                glob.glob(os.path.join(parent_dir, sub_dir,
                                       file_ext))):  # 遍历数据集的所有文件

            # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,
                                                          sr=sample_rate).T,
                           axis=0)  # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])

    return [feature, label]

In [ ]:
# 全部数据参与训练
parent_dir = '/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/train'
save_dir = "/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/"
folds = sub_dirs = np.array([
    'aloe', 'burger', 'cabbage', 'candied_fruits', 'carrots', 'chips',
    'chocolate', 'drinks', 'fries', 'grapes', 'gummies', 'ice-cream', 'jelly',
    'noodles', 'pickles', 'pizza', 'ribs', 'salmon', 'soup', 'wings'
])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir, sub_dirs)

 61%|██████    | 273/446 [00:12<00:08, 20.52it/s]

In [ ]:
temp = np.array(temp)
data = temp.transpose()

In [ ]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：', X.shape)
print('Y的特征尺寸是：', Y.shape)

In [11]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [12]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(2000, 128)
(2000, 20)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    random_state=1,
                                                    stratify=Y)
print('训练集的大小', len(X_train))
print('测试集的大小', len(X_test))

训练集的大小 1500
测试集的大小 500


In [14]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

参考：https://www.kaggle.com/leonadoice/task4

In [15]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(
    Conv2D(64, (3, 3),
           padding="same",
           activation="tanh",
           input_shape=input_dim))  # 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))  # 最大池化
model.add(Conv2D(128, (3, 3), padding="same", activation="tanh"))  #卷积层
model.add(MaxPool2D(pool_size=(2, 2)))  # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten())  # 展开
model.add(Dense(1024, activation="tanh"))
model.add(Dense(20, activation="softmax"))  # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [18]:
# 训练模型
model.fit(X_train,
          Y_train,
          epochs=50,
          batch_size=15,
          validation_data=(X_test, Y_test))

Epoch 1/50
100/100 [==============================] - 1s 11ms/step - loss: 0.3862 - accuracy: 0.8807 - val_loss: 3.2746 - val_accuracy: 0.4900
Epoch 2/50
100/100 [==============================] - 1s 11ms/step - loss: 0.3704 - accuracy: 0.8873 - val_loss: 3.5357 - val_accuracy: 0.4660
Epoch 3/50
100/100 [==============================] - 1s 10ms/step - loss: 0.4014 - accuracy: 0.8880 - val_loss: 3.4064 - val_accuracy: 0.4920
Epoch 4/50
100/100 [==============================] - 1s 10ms/step - loss: 0.3127 - accuracy: 0.9080 - val_loss: 3.5684 - val_accuracy: 0.4760
Epoch 5/50
100/100 [==============================] - 1s 11ms/step - loss: 0.2585 - accuracy: 0.9240 - val_loss: 3.8338 - val_accuracy: 0.4480
Epoch 6/50
100/100 [==============================] - 1s 11ms/step - loss: 0.3874 - accuracy: 0.8840 - val_loss: 3.9268 - val_accuracy: 0.4560
Epoch 7/50
100/100 [==============================] - 1s 11ms/step - loss: 0.3469 - accuracy: 0.9007 - val_loss: 3.8774 - val_accuracy: 0.4740

### 预测测试集

In [19]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir,
                                          file_ext))[:]):  # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,
                       axis=0)  # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [20]:
X_test = extract_features('/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/')

100%|██████████| 2000/2000 [01:30<00:00, 22.17it/s]


In [22]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [24]:
preds = np.argmax(predictions, axis=1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/*.wav')
result = pd.DataFrame({'name': path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv', index=None)

In [25]:
!ls /Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/*.wav | wc -l

    2000


In [26]:
!wc -l submit.csv

    2001 submit.csv


In [27]:
!pwd

/Users/sampras/Desktop/待学习/NLP/Task1 食物声音识别之Baseline学习/Task1 CNN_Baseline_1
